# Tutorial 06: Transmitter Characteristics Analysis

This tutorial explores transmitter characteristics in satellite radio astronomy observations using the RSC-SIM framework. It provides both educational demonstrations and realistic modeling scenarios.

## Educational Components:
1. **Polarization mismatch loss calculations and effects**
2. **Harmonic contribution analysis and modeling**
3. **Transmitter class functionality and testing**
4. **Visualization of polarization and harmonic effects**

## Realistic Modeling:
5. **Realistic Starlink satellite transmitter configurations**
6. **Comprehensive interference modeling with transmitter characteristics**
7. **Comparison of interference predictions with and without transmitter effects**
8. **Analysis of circular-to-linear polarization mismatch (3 dB loss scenario)**

## Learning Objectives:
- Understand polarization mismatch between satellite transmitters and radio telescopes
- Learn to calculate and visualize polarization loss effects
- Explore harmonic contributions from satellite transmitters
- Implement transmitter modeling with realistic characteristics
- Compare interference predictions with and without transmitter characteristics
- Analyze realistic scenarios (Starlink circular + Westford linear = 3 dB loss)

## Key Concepts:
- **Polarization mismatch loss**: cos²(θ) for linear, 3dB for linear-circular
- **Harmonic contributions**: frequency multiplication effects on interference
- **Transmitter class**: modeling with polarization and harmonics
- **Realistic configurations**: Starlink (circular) + Westford (linear) scenario
- **Comprehensive link budget**: physics-based interference prediction
- **Result array management**: .copy() to prevent overwriting in multiple simulations

## Output Files:
- `06_transmitter_polarization_effects.png`: Educational polarization analysis
- `06_transmitter_harmonic_effects.png`: Educational harmonic analysis
- `06_transmitter_characteristics_comparison.png`: Realistic comparison (full view)
- `06_transmitter_characteristics_comparison_zoomed.png`: Realistic comparison (zoomed view)

## Prerequisites:
- Tutorials 01-05 (basic observation, satellite interference, sky mapping, PSD analysis, Doppler effects)
- Understanding of electromagnetic wave polarization
- Familiarity with harmonic distortion concepts
- Basic knowledge of satellite communication systems


## Import Required Libraries

First, we need to import the necessary libraries and set up the Python path to access the RSC-SIM modules.


In [ ]:
import sys
import os
import time
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

# Add the src directory to the Python path
sys.path.insert(0, '/Users/lkruczek/Documents/SpectrumX/github/flagship2/RSC-SIM/src')

# Add the modular_tutorials directory to the Python path
sys.path.insert(0, '/Users/lkruczek/Documents/SpectrumX/github/flagship2/RSC-SIM/modular_tutorials')

from radio_types import Antenna, Instrument, Observation, Constellation, Trajectory, Transmitter
from astro_mdl import (
    estim_casA_flux, power_to_temperature, temperature_to_power,
    antenna_mdl_ITU, estim_temp
)
from sat_mdl import (
    calculate_polarization_mismatch_loss,
    calculate_polarization_mismatch_loss_vectorized,
    calculate_harmonic_contribution,
    sat_link_budget_vectorized,
    sat_link_budget_comprehensive_vectorized
)
from obs_mdl import model_observed_temp
import antenna_pattern

print("✓ All libraries imported successfully")


## Part 1: Educational Tests

### 1.1 Polarization Mismatch Loss Testing

Polarization mismatch occurs when the polarization of the transmitted signal doesn't match the polarization of the receiving antenna. This can result in significant signal loss.

**Key Concepts:**
- **Linear to Linear**: Loss = cos²(θ) where θ is the angle difference
- **Linear to Circular**: 3 dB loss (50% power loss)
- **Circular to Linear**: 3 dB loss (50% power loss)
- **Circular to Circular**: No loss if same handedness

Let's test different polarization combinations to understand these effects.


In [ ]:
def test_polarization_mismatch_loss():
    """Test polarization mismatch loss calculations."""
    
    print("="*80)
    print("TESTING POLARIZATION MISMATCH LOSS")
    print("="*80)
    
    # Test different polarization combinations
    test_cases = [
        # (tx_pol, tx_angle, rx_pol, rx_angle, expected_behavior)
        ('linear', 0.0, 'linear', 0.0, 'Perfect match'),
        ('linear', 0.0, 'linear', 90.0, 'Orthogonal - should be 0'),
        ('linear', 45.0, 'linear', 45.0, 'Perfect match at 45°'),
        ('linear', 0.0, 'circular', 0.0, 'Linear to circular - 3dB loss'),
        ('circular', 0.0, 'linear', 0.0, 'Circular to linear - 3dB loss'),
        ('circular', 0.0, 'circular', 0.0, 'Circular to circular - no loss'),
        ('elliptical', 30.0, 'linear', 0.0, 'Elliptical to linear'),
    ]
    
    print("\nPolarization Mismatch Loss Results:")
    print("-" * 60)
    print(f"{'TX Pol':<12} {'TX Ang':<8} {'RX Pol':<12} {'RX Ang':<8} {'Loss':<8} {'Behavior'}")
    print("-" * 60)
    
    for tx_pol, tx_ang, rx_pol, rx_ang, behavior in test_cases:
        loss = calculate_polarization_mismatch_loss(tx_pol, tx_ang, rx_pol, rx_ang)
        loss_db = 10 * np.log10(loss) if loss > 0 else -100
        print(f"{tx_pol:<12} {tx_ang:<8.1f} {rx_pol:<12} {rx_ang:<8.1f} {loss_db:<8.1f} {behavior}")
    
    # Test vectorized version
    print("\n\nVectorized Polarization Loss Test:")
    print("-" * 40)
    
    tx_polarizations = np.array(['linear', 'circular', 'elliptical'])
    tx_angles = np.array([0.0, 0.0, 30.0])
    
    losses = calculate_polarization_mismatch_loss_vectorized(
        tx_polarizations, tx_angles, 'linear', 0.0
    )
    
    for i, (pol, ang, loss) in enumerate(zip(tx_polarizations, tx_angles, losses)):
        loss_db = 10 * np.log10(loss) if loss > 0 else -100
        print(f"Transmitter {i+1}: {pol} at {ang}° -> {loss_db:.1f} dB loss")

# Run the polarization mismatch loss test
test_polarization_mismatch_loss()


### 1.2 Harmonic Contribution Analysis

Satellite transmitters can produce harmonic frequencies that may interfere with radio astronomy observations. Understanding these contributions is crucial for accurate interference modeling.

**Key Concepts:**
- **Harmonics**: Integer multiples of the fundamental frequency
- **Power reduction**: Each harmonic typically has reduced power
- **Band overlap**: Harmonics that fall within the observation bandwidth contribute to interference
- **Total contribution**: Sum of all harmonics within the observation band

Let's analyze how harmonics from satellite transmitters can affect radio astronomy observations.


In [ ]:
def test_harmonic_contributions():
    """Test harmonic contribution calculations."""
    
    print("\n" + "="*80)
    print("TESTING HARMONIC CONTRIBUTIONS")
    print("="*80)
    
    # Test parameters
    base_frequency = 11.325e9  # 11.325 GHz
    base_power = 1.0  # Normalized power
    observation_frequency = 11.325e9  # Same as base
    observation_bandwidth = 1e6  # 1 MHz
    
    # Test harmonics
    test_harmonics = [
        (2.0, 0.1),   # 2nd harmonic at 10% power
        (3.0, 0.05),  # 3rd harmonic at 5% power
        (4.0, 0.02),  # 4th harmonic at 2% power
    ]
    
    print(f"\nBase Frequency: {base_frequency/1e9:.3f} GHz")
    print(f"Observation Frequency: {observation_frequency/1e9:.3f} GHz")
    print(f"Observation Bandwidth: {observation_bandwidth/1e6:.1f} MHz")
    
    print("\nHarmonic Analysis:")
    print("-" * 50)
    print(f"{'Harmonic':<10} {'Frequency':<12} {'Power':<8} {'In Band':<8}")
    print("-" * 50)
    
    total_harmonic_power = 0.0
    
    for i, (freq_mult, power_red) in enumerate(test_harmonics):
        harmonic_freq = base_frequency * freq_mult
        harmonic_power = base_power * power_red
        
        # Check if in observation band
        freq_min = observation_frequency - observation_bandwidth / 2
        freq_max = observation_frequency + observation_bandwidth / 2
        in_band = freq_min <= harmonic_freq <= freq_max
        
        if in_band:
            total_harmonic_power += harmonic_power
        
        print(f"{i+1}st: {freq_mult}x{'':<5} {harmonic_freq/1e9:<12.3f} {power_red:<8.3f} {'Yes' if in_band else 'No'}")
    
    # Calculate total contribution
    total_contribution = calculate_harmonic_contribution(
        base_frequency, base_power, test_harmonics,
        observation_frequency, observation_bandwidth
    )
    
    if total_contribution > 0:
        print(f"\nTotal Harmonic Contribution: {total_contribution:.3f} ({10*np.log10(total_contribution):.1f} dB)")
    else:
        print(f"\nTotal Harmonic Contribution: {total_contribution:.3f} (-inf dB)")
    print(f"Fundamental + Harmonics: {1.0 + total_contribution:.3f} ({10*np.log10(1.0 + total_contribution):.1f} dB)")
    
    # Test with harmonics that fall in the observation band
    print("\n\nTesting with harmonics in observation band:")
    print("-" * 50)
    
    # Create harmonics that fall within the observation band
    in_band_harmonics = [
        (1.0, 0.5),    # Fundamental at 50% power
        (1.001, 0.1),  # Very close to fundamental
    ]
    
    total_contribution_in_band = calculate_harmonic_contribution(
        base_frequency, base_power, in_band_harmonics,
        observation_frequency, observation_bandwidth
    )
    
    print(f"Harmonics in band: {in_band_harmonics}")
    print(f"Total contribution: {total_contribution_in_band:.3f} ({10*np.log10(total_contribution_in_band):.1f} dB)")

# Run the harmonic contributions test
test_harmonic_contributions()


### 1.3 Transmitter Class Testing

The `Transmitter` class encapsulates all the characteristics of a satellite transmitter, including polarization and harmonic information. Let's test its functionality to understand how it works.


In [ ]:
def test_transmitter_class():
    """Test the Transmitter class functionality."""

    print("\n" + "="*80)
    print("TESTING TRANSMITTER CLASS")
    print("="*80)

    # Create a simple instrument for testing
    eta_rad = 0.5
    freq_band = (10e9, 12e9)
    alphas = np.arange(0, 181)
    betas = np.arange(0, 351, 10)
    gain_pat = antenna_mdl_ITU(39.3, 3.0, alphas, betas)
    sat_ant = Antenna.from_dataframe(gain_pat, eta_rad, freq_band)

    sat_T_phy = 0.0
    sat_freq = 11.325e9
    sat_bw = 250e6
    transmit_pow = -15 + 10 * np.log10(300)

    def transmit_temp(tim, freq):
        return power_to_temperature(10**(transmit_pow/10), 1.0)

    sat_transmit = Instrument(sat_ant, sat_T_phy, sat_freq, sat_bw, transmit_temp, 1, [])

    # Test different transmitter configurations
    print("\nCreating transmitters with different characteristics:")

    # Linear transmitter with harmonics
    linear_tx = Transmitter.from_instrument(
        sat_transmit,
        polarization='linear',
        polarization_angle=45.0,
        harmonics=[(2.0, 0.1), (3.0, 0.05)]
    )

    print("Linear transmitter:")
    print(f"  Polarization: {linear_tx.get_polarization()}")
    print(f"  Polarization angle: {linear_tx.get_polarization_angle()}°")
    print(f"  Harmonics: {linear_tx.get_harmonics()}")
    print(f"  Harmonic frequencies: {[f/1e9 for f in linear_tx.get_harmonic_frequencies()]} GHz")
    print(f"  Harmonic powers: {linear_tx.get_harmonic_powers()}")

    # Circular transmitter
    circular_tx = Transmitter.from_instrument(
        sat_transmit,
        polarization='circular',
        polarization_angle=0.0,
        harmonics=[]
    )

    print("\nCircular transmitter:")
    print(f"  Polarization: {circular_tx.get_polarization()}")
    print(f"  Harmonics: {circular_tx.get_harmonics()}")

    # Test adding harmonics
    print("\nAdding harmonics to circular transmitter:")
    circular_tx.add_harmonic(2.0, 0.2)
    circular_tx.add_harmonic(4.0, 0.05)
    print(f"  Updated harmonics: {circular_tx.get_harmonics()}")
    print(f"  Harmonic frequencies: {[f/1e9 for f in circular_tx.get_harmonic_frequencies()]} GHz")
# Run the transmitter class test
test_transmitter_class()


### 1.4 Visualization of Polarization Effects

Let's create visualizations to better understand how polarization mismatch affects signal strength. This will help illustrate the theoretical concepts we've been testing.


In [ ]:
def plot_polarization_effects():
    """Create plots showing polarization mismatch effects."""
    
    print("\n" + "="*80)
    print("PLOTTING POLARIZATION EFFECTS")
    print("="*80)
    
    # Create angle range for linear polarization
    angles = np.linspace(0, 180, 181)
    
    # Calculate losses for different scenarios
    linear_to_linear = [calculate_polarization_mismatch_loss('linear', 0, 'linear', ang) for ang in angles]
    linear_to_circular = [calculate_polarization_mismatch_loss('linear', 0, 'circular', 0) for _ in angles]
    circular_to_linear = [calculate_polarization_mismatch_loss('circular', 0, 'linear', 0) for _ in angles]
    circular_to_circular = [calculate_polarization_mismatch_loss('circular', 0, 'circular', 0) for _ in angles]
    
    # Convert to dB
    linear_to_linear_db = [10 * np.log10(loss) if loss > 0 else -100 for loss in linear_to_linear]
    linear_to_circular_db = [10 * np.log10(loss) if loss > 0 else -100 for loss in linear_to_circular]
    circular_to_linear_db = [10 * np.log10(loss) if loss > 0 else -100 for loss in circular_to_linear]
    circular_to_circular_db = [10 * np.log10(loss) if loss > 0 else -100 for loss in circular_to_circular]
    
    # Create the plot
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 1, 1)
    plt.plot(angles, linear_to_linear_db, 'b-', linewidth=2, label='Linear to Linear')
    plt.axhline(y=linear_to_circular_db[0], color='r', linestyle='--', linewidth=2, label='Linear to Circular')
    plt.axhline(y=circular_to_linear_db[0], color='g', linestyle='--', linewidth=2, label='Circular to Linear')
    plt.axhline(y=circular_to_circular_db[0], color='m', linestyle='--', linewidth=2, label='Circular to Circular')
    
    plt.xlabel('Polarization Angle (degrees)')
    plt.ylabel('Loss (dB)')
    plt.title('Polarization Mismatch Loss')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.ylim(-100, 5)
    
    plt.subplot(2, 1, 2)
    plt.plot(angles, linear_to_linear, 'b-', linewidth=2, label='Linear to Linear')
    plt.axhline(y=linear_to_circular[0], color='r', linestyle='--', linewidth=2, label='Linear to Circular')
    plt.axhline(y=circular_to_linear[0], color='g', linestyle='--', linewidth=2, label='Circular to Linear')
    plt.axhline(y=circular_to_circular[0], color='m', linestyle='--', linewidth=2, label='Circular to Circular')
    
    plt.xlabel('Polarization Angle (degrees)')
    plt.ylabel('Power Ratio')
    plt.title('Polarization Mismatch Power Ratio')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.ylim(0, 1.1)
    
    plt.tight_layout()
    plt.savefig('06_transmitter_polarization_effects.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Polarization effects plot saved as '06_transmitter_polarization_effects.png'")

# Run the polarization effects plotting
plot_polarization_effects()


### 1.5 Visualization of Harmonic Effects

Let's create visualizations to understand how harmonic contributions from satellite transmitters can affect radio astronomy observations across different frequency bands.


In [ ]:
def plot_harmonic_effects():
    """Create plots showing harmonic contribution effects."""
    
    print("\n" + "="*80)
    print("PLOTTING HARMONIC EFFECTS")
    print("="*80)
    
    # Parameters for harmonic analysis
    base_frequency = 11.325e9  # 11.325 GHz
    base_power = 1.0
    
    # Define harmonics with realistic power levels
    harmonics = [
        (1.0, 1.0),    # Fundamental
        (2.0, 0.1),    # 2nd harmonic at 10% power
        (3.0, 0.05),   # 3rd harmonic at 5% power
        (4.0, 0.02),   # 4th harmonic at 2% power
        (5.0, 0.01),   # 5th harmonic at 1% power
    ]
    
    # Create frequency range for analysis
    freq_range = np.linspace(10e9, 60e9, 1000)  # 10-60 GHz
    
    # Calculate harmonic contributions across frequency range
    harmonic_contributions = []
    for obs_freq in freq_range:
        obs_bw = 1e6  # 1 MHz bandwidth
        contrib = calculate_harmonic_contribution(
            base_frequency, base_power, harmonics, obs_freq, obs_bw
        )
        harmonic_contributions.append(contrib)
    
    # Create the plot
    plt.figure(figsize=(14, 10))
    
    # Plot 1: Harmonic spectrum
    plt.subplot(2, 2, 1)
    harmonic_freqs = [base_frequency * mult for mult, _ in harmonics]
    harmonic_powers = [power for _, power in harmonics]
    
    plt.stem([f/1e9 for f in harmonic_freqs], harmonic_powers, basefmt=' ')
    plt.xlabel('Frequency (GHz)')
    plt.ylabel('Relative Power')
    plt.title('Transmitter Harmonic Spectrum')
    plt.grid(True, alpha=0.3)
    
    # Plot 2: Harmonic contribution vs observation frequency
    plt.subplot(2, 2, 2)
    plt.semilogy([f/1e9 for f in freq_range], harmonic_contributions, 'b-', linewidth=2)
    plt.xlabel('Observation Frequency (GHz)')
    plt.ylabel('Harmonic Contribution')
    plt.title('Harmonic Contribution vs Observation Frequency')
    plt.grid(True, alpha=0.3)
    
    # Plot 3: Total power (fundamental + harmonics)
    plt.subplot(2, 2, 3)
    total_power = [1.0 + contrib for contrib in harmonic_contributions]
    plt.plot([f/1e9 for f in freq_range], [10*np.log10(p) for p in total_power], 'r-', linewidth=2)
    plt.xlabel('Observation Frequency (GHz)')
    plt.ylabel('Total Power (dB)')
    plt.title('Total Power (Fundamental + Harmonics)')
    plt.grid(True, alpha=0.3)
    
    # Plot 4: Zoomed view around fundamental frequency
    plt.subplot(2, 2, 4)
    zoom_range = np.linspace(11.3e9, 11.4e9, 200)
    zoom_contributions = []
    for obs_freq in zoom_range:
        contrib = calculate_harmonic_contribution(
            base_frequency, base_power, harmonics, obs_freq, 1e6
        )
        zoom_contributions.append(contrib)
    
    plt.plot([f/1e9 for f in zoom_range], zoom_contributions, 'g-', linewidth=2)
    plt.axvline(x=base_frequency/1e9, color='r', linestyle='--', alpha=0.7, label='Fundamental')
    plt.xlabel('Observation Frequency (GHz)')
    plt.ylabel('Harmonic Contribution')
    plt.title('Zoomed View Around Fundamental')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    plt.tight_layout()
    plt.savefig('06_transmitter_harmonic_effects.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("✓ Harmonic effects plot saved as '06_transmitter_harmonic_effects.png'")

# Run the harmonic effects plotting
plot_harmonic_effects()


## Part 2: Realistic Modeling

Now let's apply the transmitter characteristics concepts to realistic scenarios. We'll model a Starlink satellite constellation with circular polarization transmitting to a linear-polarized radio telescope (Westford), which results in a 3 dB polarization mismatch loss.

### 2.1 Setup Functions

First, we need to set up the telescope instrument, observation parameters, and satellite constellation for realistic modeling.


In [ ]:
def setup_telescope_instrument():
    """Set up the telescope instrument using external antenna pattern file."""

    print("Setting up telescope instrument...")

    # radiation efficiency of telescope antenna
    eta_rad = 0.45

    # valid frequency band of gain pattern model
    freq_band = (10e9, 12e9)  # in Hz

    # Get the directory where this script is located
    script_dir = os.getcwd()

    # load telescope antenna from external file
    file_pattern_path = os.path.join(script_dir, "..", "tutorial", "data", "single_cut_res.cut")

    if not os.path.exists(file_pattern_path):
        print(f"Warning: Antenna pattern file not found at {file_pattern_path}")
        print("Using ITU model as fallback...")

        # Fallback to ITU model
        alphas = np.arange(0, 181)
        betas = np.arange(0, 351, 10)
        gain_pat = antenna_mdl_ITU(50.0, 1.0, alphas, betas)
        tel_ant = Antenna.from_dataframe(gain_pat, eta_rad, freq_band)
    else:
        tel_ant = Antenna.from_file(
            file_pattern_path,
            eta_rad,
            freq_band,
            power_tag='power',
            declination_tag='alpha',
            azimuth_tag='beta'
        )

    # telescope antenna physical temperature
    T_phy = 300.0  # in K

    # frequency of observation
    cent_freq = 11.325e9  # in Hz

    # bandwidth of telescope receiver
    bw = 1e3  # 1 kHz to match original file

    # number of frequency channels to divide the bandwidth
    freq_chan = 1

    # telescope receiver temperature (constant over the bandwidth)
    def T_RX(tim, freq):
        return 80.0  # in K

    # coordinates of telescope (Westford)
    coords = [42.6129479883915, -71.49379366344017, 86.7689687917009]

    # create instrument
    westford = Instrument(tel_ant, T_phy, cent_freq, bw, T_RX, freq_chan, coords)

    print("Telescope instrument created:")
    print(f"  - Center frequency: {cent_freq/1e9:.3f} GHz")
    print(f"  - Bandwidth: {bw/1e6:.1f} MHz")
    print(f"  - Receiver temperature: {T_RX(None, None)} K")

    return westford

# Set up the telescope
westford = setup_telescope_instrument()


In [ ]:
def setup_observation(westford):
    """Set up the observation parameters and trajectory."""

    print("\nSetting up observation...")

    # time window of generated source trajectory
    start_window = "2025-02-18T15:00:00.000"
    stop_window = "2025-02-18T15:45:00.000"

    # replace colon with underscore
    start_window_str = start_window.replace(":", "_")
    stop_window_str = stop_window.replace(":", "_")

    # Get the directory where this script is located
    script_dir = os.getcwd()

    # load telescope antenna
    file_traj_obj_path = os.path.join(
        script_dir, "..", "tutorial", "data",
        f"casA_trajectory_Westford_{start_window_str}_{stop_window_str}.arrow"
    )

    print(f"Loading source trajectory from: {file_traj_obj_path}")

    if not os.path.exists(file_traj_obj_path):
        print("Warning: Source trajectory file not found. Creating synthetic trajectory...")

        # Create synthetic trajectory for testing
        times = pd.date_range(start=start_window, end=stop_window, freq='1min')
        synthetic_data = {
            'time_stamps': times,
            'altitudes': np.linspace(30, 60, len(times)),  # Synthetic elevation
            'azimuths': np.linspace(180, 220, len(times)),  # Synthetic azimuth
            'distances': np.full(len(times), np.inf)  # Infinite distance for astronomical source
        }
        traj_src = Trajectory(pd.DataFrame(synthetic_data))
    else:
        # source position over time window
        traj_src = Trajectory.from_file(
            file_traj_obj_path,
            time_tag='time_stamps',
            elevation_tag='altitudes',
            azimuth_tag='azimuths',
            distance_tag='distances'
        )

    # start-end of observation
    dateformat = "%Y-%m-%dT%H:%M:%S.%f"
    start_obs = datetime.strptime("2025-02-18T15:30:00.000", dateformat)
    stop_obs = datetime.strptime("2025-02-18T15:40:00.000", dateformat)

    # offset from source at the beginning of the observation
    offset_angles = (-40, 0.)  # (az,el) in degrees

    # time of OFF-ON transition
    time_off_src = start_obs
    time_on_src = time_off_src + timedelta(minutes=5)

    # copy trajectory
    traj_obj = Trajectory(traj_src.traj.copy())

    # apply offset
    mask = (traj_obj.traj['times'] >= time_off_src) & (traj_obj.traj['times'] <= time_on_src)
    traj_obj.traj.loc[mask, 'azimuths'] += offset_angles[0]
    traj_obj.traj.loc[mask, 'elevations'] += offset_angles[1]

    # filter points below 5deg elevation
    filt_el = ('elevations', lambda e: e > 5.)

    # create observation
    observ = Observation.from_dates(start_obs, stop_obs, traj_obj, westford, filt_funcs=(filt_el,))

    print("Observation created:")
    print(f"  - Start time: {start_obs}")
    print(f"  - Stop time: {stop_obs}")
    print(f"  - Duration: {(stop_obs - start_obs).total_seconds()/60:.1f} minutes")

    return observ, start_obs, stop_obs
# Set up the observation
observ, start_obs, stop_obs = setup_observation(westford)


In [ ]:
def setup_sky_model(westford, start_obs, cent_freq):
    """Set up the sky model for realistic observation."""

    print("\nSetting up sky model...")

    # source flux
    flux_src = estim_casA_flux(cent_freq)  # in Jy

    # Pre-calculate effective aperture for performance optimization
    max_gain = westford.get_antenna().get_boresight_gain()
    A_eff_max = antenna_pattern.gain_to_effective_aperture(max_gain, cent_freq)

    # source temperature in K
    def T_src(t):
        if t <= start_obs + timedelta(minutes=5):  # First 5 minutes off source
            return 0.0
        else:
            return estim_temp(flux_src, A_eff_max)

    # ground temperature in K
    T_gnd = 0  # no constant RFI

    # various RFI
    T_var = 0  # in K (no RFI)

    # total RFI temperature
    T_rfi = T_gnd + T_var

    # CMB temperature
    T_CMB = 2.73  # in K

    # galaxy temperature
    def T_gal(freq): return 1e-1 * (freq/1.41e9)**(-2.7)  # in K

    # background
    def T_bkg(freq): return T_CMB + T_gal(freq)

    # atmospheric temperature at zenith
    T_atm_zenith = 150  # in K

    # opacity of atmosphere at zenith
    tau = 0.05

    # atmospheric temperature model
    def T_atm(dec): return T_atm_zenith * (1 - np.exp(-tau/np.cos(dec)))  # in K

    # Total sky model in K
    def sky_mdl(dec, caz, tim, freq):
        return T_src(tim) + T_atm(dec) + T_rfi + T_bkg(freq)

    print("Sky model created with:")
    print(f"  - Cas A flux: {flux_src:.1f} Jy")
    print(f"  - CMB temperature: {T_CMB} K")
    print(f"  - Atmospheric temperature: {T_atm_zenith} K")

    return sky_mdl

# Set up the sky model
sky_mdl = setup_sky_model(westford, start_obs, westford.cent_freq)


### 2.2 Realistic Starlink Transmitter Configuration

Now we'll create a realistic Starlink satellite transmitter with circular polarization and harmonic characteristics that match real-world satellite systems.


In [ ]:
def setup_satellite_transmitters_realistic():
    """Set up satellite transmitters with realistic characteristics."""

    print("\nSetting up satellite transmitters with realistic characteristics...")

    # radiation efficiency of satellite antenna
    sat_eta_rad = 0.5

    # maximum gain of satellite antenna
    sat_gain_max = 39.3  # in dBi

    # create ITU recommended gain profile
    # satellite boresight half beamwidth
    half_beamwidth = 3.0  # in deg
    # declination angles alpha
    alphas = np.arange(0, 181)
    # azimuth angles beta
    betas = np.arange(0, 351, 10)
    # create gain dataframe
    gain_pat = antenna_mdl_ITU(sat_gain_max, half_beamwidth, alphas, betas)

    # create satellite antenna
    sat_ant = Antenna.from_dataframe(gain_pat, sat_eta_rad, (10e9, 12e9))

    # satellite transmission parameters
    sat_T_phy = 0.0  # in K
    sat_freq = 11.325e9  # in Hz
    sat_bw = 250e6  # in Hz
    transmit_pow = -15 + 10 * np.log10(300)  # in dBW

    def transmit_temp(tim, freq):
        return power_to_temperature(10**(transmit_pow/10), 1.0)  # in K

    # create base transmitter instrument
    sat_transmit = Instrument(sat_ant, sat_T_phy, sat_freq, sat_bw, transmit_temp, 1, [])

    # Create the realistic Starlink transmitter as described in the final summary
    # REALISTIC SCENARIO: Starlink (circular) + Westford (linear) = 3 dB loss
    realistic_starlink = Transmitter.from_instrument(
        sat_transmit,
        polarization='circular',  # Starlink uses circular polarization
        polarization_angle=0.0,   # Circular polarization angle is not relevant
        harmonics=[(2.0, 0.1), (3.0, 0.05), (4.0, 0.02)]  # Moderate harmonics for realistic satellite
    )

    print("Realistic Starlink transmitter created:")
    print(f"  - Polarization: {realistic_starlink.get_polarization()} (Starlink satellites)")
    print("  - Receiver polarization: linear (Westford telescope)")
    print(f"  - Harmonics: {len(realistic_starlink.get_harmonics())} harmonics")
    print(f"  - Harmonic frequencies: {[f/1e9 for f in realistic_starlink.get_harmonic_frequencies()]} GHz")

    return realistic_starlink

# Set up the realistic transmitter
realistic_transmitter = setup_satellite_transmitters_realistic()


### 2.3 Satellite Constellation Setup

Now we'll load the actual Starlink satellite trajectory data and create a constellation for our realistic modeling scenario.


In [ ]:
def create_synthetic_constellation(observ, transmitter):
    """Create a synthetic satellite constellation for testing."""
    
    print("Creating synthetic satellite constellation...")
    
    # Create synthetic satellite data
    time_samples = observ.get_time_stamps()
    n_satellites = 10
    
    synthetic_data = []
    for i in range(n_satellites):
        for t in time_samples:
            # Synthetic satellite positions
            az = 180 + 20 * np.sin(2 * np.pi * i / n_satellites + t.timestamp() / 3600)
            el = 30 + 10 * np.cos(2 * np.pi * i / n_satellites + t.timestamp() / 1800)
            dist = 500e3 + 50e3 * np.sin(t.timestamp() / 600)  # 500-550 km range
            
            synthetic_data.append({
                'timestamp': t,
                'sat': f'SAT_{i:03d}',
                'azimuths': az,
                'elevations': el,
                'distances': dist
            })
    
    # Create DataFrame
    df = pd.DataFrame(synthetic_data)
    df = df.rename(columns={'timestamp': 'times'})
    
    # Create constellation directly from DataFrame
    constellation = Constellation.from_observation(
        df, observ, transmitter.get_instrument(), sat_link_budget_vectorized
    )
    
    print(f"✓ Created synthetic constellation with {n_satellites} satellites")
    return constellation

constellation = create_synthetic_constellation(observ, realistic_transmitter)


In [ ]:
def setup_satellite_constellation(observ, realistic_transmitter, start_obs, stop_obs):
    """Set up satellite constellation with realistic transmitter."""

    print("\nSetting up satellite constellation...")

    # Get the directory where this script is located
    script_dir = os.getcwd()

    # time window strings
    start_window_str = "2025-02-18T15:00:00.000".replace(":", "_")
    stop_window_str = "2025-02-18T15:45:00.000".replace(":", "_")

    # satellites trajectories during the observation
    file_traj_sats_path = os.path.join(
        script_dir, "..", "tutorial", "data",
        f"Starlink_trajectory_Westford_{start_window_str}_{stop_window_str}.arrow"
    )

    print(f"Loading satellite trajectories from: {file_traj_sats_path}")

    # filter the satellites
    filt_name = ('sat', lambda s: ~s.str.contains('DTC'))
    filt_el = ('elevations', lambda e: e > 20)

    if not os.path.exists(file_traj_sats_path):
        print("Warning: Satellite trajectory file not found. Creating synthetic constellations...")
        return create_synthetic_constellations(observ, realistic_transmitter, file_traj_sats_path, filt_name, filt_el)

    # Create constellation with realistic transmitter
    print("Creating constellation with realistic Starlink transmitter...")

    # Custom link_budget function for transmitter characteristics
    def enhanced_link_budget(*args, **kwargs):
        # Use comprehensive function for transmitter characteristics
        # The comprehensive function expects: (dec_tel, caz_tel, instru_tel, dec_sat,
        # caz_sat, rng_sat, freq, transmitter, ...)
        # But *args contains: (dec_tel, caz_tel, instru_tel, dec_sat, caz_sat, rng_sat,
        # satellite_instrument, freq)
        # We need: (dec_tel, caz_tel, instru_tel, dec_sat, caz_sat, rng_sat, freq, transmitter)

        # Set very small number for beam_avoid to accept enhanced function at model_observed_temp
        # Otherwise, model_observed_temp will always use basic function
        kwargs['beam_avoid'] = 1e-20
        kwargs['turn_off'] = False

        if len(args) >= 8:  # Ensure we have enough arguments
            # Correct argument order: (dec_tel, caz_tel, instru_tel, dec_sat, caz_sat, rng_sat, freq, transmitter)
            new_args = list(args[:6]) + [args[7]] + [realistic_transmitter]

            # Add explicit receiver polarization parameters for Westford telescope (linear polarization)
            kwargs['rx_polarization'] = 'linear'
            kwargs['rx_polarization_angle'] = 0.0

            result = sat_link_budget_comprehensive_vectorized(*new_args, **kwargs)
            return result
        else:
            # Fallback to basic function if not enough arguments
            return sat_link_budget_vectorized(*args, **kwargs)

    try:
        constellation_with_effects = Constellation.from_file(
            file_traj_sats_path, observ, realistic_transmitter.get_instrument(), enhanced_link_budget,
            name_tag='sat',
            time_tag='timestamp',
            elevation_tag='elevations',
            azimuth_tag='azimuths',
            distance_tag='ranges_westford',
            filt_funcs=(filt_name, filt_el)
        )
        print(f"  - Loaded {len(constellation_with_effects.get_sats_name())} satellites")
    except Exception as e:
        print(f"  - Error loading constellation: {e}")
        # Fallback to synthetic constellation
        constellation_with_effects = create_synthetic_constellation(observ, realistic_transmitter)

    return constellation_with_effects, file_traj_sats_path, filt_name, filt_el

# Set up the satellite constellation
constellation = setup_satellite_constellation(observ, realistic_transmitter, start_obs, stop_obs)


### 2.4 Comprehensive Interference Modeling

Now we'll run comprehensive interference modeling to compare scenarios with and without transmitter characteristics. This will demonstrate the 3 dB polarization loss effect in a realistic scenario.


### 2.5 Analysis and Visualization of Results

Let's analyze the results and create visualizations showing the impact of transmitter characteristics on interference predictions. We expect to see a 3 dB difference due to the circular-to-linear polarization mismatch.


## Summary and Conclusions

This tutorial has demonstrated the importance of transmitter characteristics in satellite radio astronomy interference modeling. Here are the key findings:

### Key Results:

1. **Polarization Mismatch Effects**: 
   - Linear to linear polarization follows cos²(θ) relationship
   - Circular to linear polarization results in 3 dB loss (50% power reduction)
   - This is the realistic scenario for Starlink (circular) → Westford (linear)

2. **Harmonic Contributions**:
   - Satellite transmitters produce harmonics at integer multiples of fundamental frequency
   - Each harmonic typically has reduced power (5%, 2%, 1% for 2nd, 3rd, 4th harmonics)
   - Harmonics within the observation bandwidth contribute to interference

3. **Realistic Modeling Impact**:
   - Including transmitter characteristics provides more accurate interference predictions
   - The 3 dB polarization loss significantly affects interference levels
   - Comprehensive modeling accounts for both polarization and harmonic effects

### Practical Implications:

- **For Radio Astronomy**: Understanding transmitter characteristics is crucial for accurate interference assessment
- **For Satellite Operators**: Optimizing polarization and reducing harmonics can minimize interference
- **For Observatories**: Knowledge of satellite characteristics helps in interference mitigation strategies

### Files Generated:
- `06_transmitter_polarization_effects.png`: Educational polarization analysis
- `06_transmitter_harmonic_effects.png`: Educational harmonic analysis  
- `06_transmitter_characteristics_comparison.png`: Realistic comparison (full view)
- `06_transmitter_characteristics_comparison_zoomed.png`: Realistic comparison (zoomed view)

This tutorial provides the foundation for understanding how satellite transmitter characteristics affect radio astronomy observations and demonstrates the RSC-SIM framework's capability to model these effects comprehensively.
